<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Financial Data Science &mdash; IO Operations

**DataNatives Berlin 2015**

Dr Yves J Hilpisch

<a href='mailto:team@tpq.io'>team@tpq.io</a> | <a href='http://tpq.io'>http://tpq.io</a>

The Python Quants GmbH

## TsTables &ndash; High Performance Times Series Management

**TsTables** is a Python library by Andy Fiedler built on top of the popular ``PyTables`` HDF5 database library. It is meant to handle large amounts of **high frequency time series data** in **append once, retrieve many times** scenarios (cf. [Gihub page](https://github.com/afiedler/tstables)). The focus lies on retrieving chunks of data from large data sets as quickly as possible.

In [ ]:
import numpy as np
import pandas as pd
import tables as tb
import tstables as tstb
import random
from time import time
from datetime import datetime 
%matplotlib inline
import seaborn as sns
sns.set()

### Sample Time Series Data

Let us generate a **decent amount of sample data points**.

In [ ]:
no = 10000000
co = 3
dt = 1. / (12 * 30 * 24 * 60)
vol = 0.2

We generate **one second intervals** of data.

In [ ]:
dr = pd.date_range('2015-1-1', periods=no, freq='1s')

In [ ]:
dr

**In memory** generation is quite quick.

In [ ]:
%%time
da = 100 * np.exp(np.cumsum(-0.5 * vol ** 2 * dt +
        vol * np.sqrt(dt) * np.random.standard_normal((no, co)), axis=0))
da[0] = 100

In [ ]:
df = pd.DataFrame(da, index=dr, columns=['ts1', 'ts2', 'ts3'])

In [ ]:
df.count()

The **starting values** of the three time series.

In [ ]:
df.head()

And a **plot** of the time series data (every 100000th point).

In [ ]:
df[::200000].plot()

### Storage and Retrieval with TsTables

To store the time series data in a **PyTables table** we first define the **table structure**.

In [ ]:
class TS(tb.IsDescription):
    timestamp = tb.Int64Col(pos=0)
    ts1 = tb.Float64Col(pos=1)
    ts2 = tb.Float64Col(pos=2)
    ts3 = tb.Float64Col(pos=3)

Second, open a database file and **create the table object**.

In [ ]:
h5 = tb.open_file('ts.h5','w')

TsTables adds a new function **``create_ts``** to PyTables.

In [ ]:
ts = h5.create_ts('/','TS', TS)

Third, we **append the time series data** to the table object.

In [ ]:
%time ts.append(df)

In [ ]:
ls -n *.h5

The **approach** of ``TsTables`` is to apply a **highly structured storage** hierarchy.

In [ ]:
a = str(h5)
print a[:508]

The strength of TsTables lies in retrieving **chunks of time series data** defined by a **start date and an end date** (which obviously is a typical case in finance, e.g. in backtesting strategies or risk management).

In [ ]:
read_start_dt = datetime(2015, 2, 1, 0, 0)
read_end_dt = datetime(2015, 2, 2, 0, 0)

TsTables tries to make such an operation **as fast as possible**.

In [ ]:
%time rows = ts.read_range(read_start_dt, read_end_dt)

Let us try it with **random intervals**.

In [ ]:
t0 = time()
its = 100
for _ in xrange(its):
    day = random.randint(1, 27)
    read_start_dt = datetime(2015, 2, day, 0, 0)
    read_end_dt = datetime(2015, 2, day + 1, 0, 0)
    rows = ts.read_range(read_start_dt, read_end_dt)
t1 = time()

The **results** are convincing.

In [ ]:
print "time for %d random accesses    %5.3f seconds" %(its, t1 - t0)
print "average time for random access  %5.3f seconds" %((t1 - t0) / its)

Conveniently, the **returned object is a pandas DataFrame**.

In [ ]:
rows.count()

In [ ]:
rows.head()

A look at a **data sub-set**.

In [ ]:
rows[::500].plot()

In [ ]:
h5.close()

In [ ]:
!rm ts.h5

## bcolz &ndash; High Performance Columnar Data Store

`bcolz` is a **columnar data store** for fast data storage and retrieval with built-in **high performance compression**. It supports both **in-memory and out-of-memory** storage and operations. Cf. <a href="http://bcolz.blosc.org/" target="_blank">http://bcolz.blosc.org/</a>.

In [ ]:
import bcolz

## ctable Example 

The first example is based on the ``ctable`` class for data in **table format**. The example data set is **100 MB** in size.

In [ ]:
N = 100000 * 100
print N

### In-Memory Storage

We generate first an **in-memory object** using high compression. Since we work with integers, **good compression ratios** are to be expected. It takes about **24 sec** to generate the ``ctable`` object from a generator via the ``fromiter`` method.

In [ ]:
%%time
ct = bcolz.fromiter(((i, i ** 2) for i in xrange(N)),
                    dtype="i4, i8",
                    count=N,
                    cparams=bcolz.cparams(clevel=9))

The **in-memory size** is about **15 MB** only, which translates in to a **compression ratio of 7+**.

In [ ]:
ct

You can now implement **fast numerical operations** on this data object (note that the output is a ``carray`` object).

In [ ]:
%time ct.eval('f0 ** 2 + sqrt(f1)')

### Disk-Based Storage

The same tasks can be implemented with **disk-based storage**. To this end, only specify the **``rootdir`` parameter**. With about **30 sec** the generation takes a bit longer on disk. everything else (especially the object handling) remaining the same however.

In [ ]:
%%time
ct = bcolz.fromiter(((i, i ** 2) for i in xrange(N)),
                    dtype="i4, i8",
                    count=N, rootdir='ct',
                    cparams=bcolz.cparams(clevel=9))

Everything else (especially the object handling) **remains almost the same** however.

In [ ]:
ct

The **numerical operations work in the same fashion** and hardly take longer due to native **multi threading and optimized caching**.

In [ ]:
%time ct.eval('f0 ** 2 + sqrt(f1)')

Let us finally verify **system disk usage**.

In [ ]:
!ls ct

In [ ]:
!du -h ct
# system disk usage

In [ ]:
!rm -r ct

## carray Example

This example is about **mid data** which does **not fit (in general) into memory** (without compression). 

In [ ]:
import numpy as np

We generte as basis a NumPy ``ndarray`` object of size **32 MB**.

In [ ]:
n = 2000
a = np.arange(n * n).reshape(n, n) 
a.nbytes

### In-Memory Storage

Let us first again work **in-memory**. Our ``carray`` object contains **400 versions** of the ``ndarray`` object. The in-memory generation of the object takes about **2.5 secs**.

In [ ]:
%%time
it = 400
ca = bcolz.carray(a, cparams=bcolz.cparams(clevel=9))
for i in range(it):
    ca.append(a)

The ``carray`` object stores **12 GB** worth of data in less than **0.1 GB** of memory, for a compression ratio of more than  **130**.

In [ ]:
ca

Let us implement the **evaluation of a numerical expression** on this data set. The syntax and handling are the same **as with NumPy ``ndarray`` objects**. 

In [ ]:
%time ca[:5000] ** 2 + np.sqrt(ca[10000:15000])

Another approach is to use the **``eval`` function** of bcolz.

In [ ]:
x = ca[:10000]  # 10,000 rows as sub-set

In [ ]:
%time bcolz.eval('x ** 2 + sqrt(x)', cparams=bcolz.cparams(clevel=9))
  # output carray object compressed

### Disk-Based Storage

**Disk-based storage** of multiple versions of the array object. We write the object **400 times** to disk in a **single ``carray`` object**. It takes only about **0.1 mins** to compress and store **12 GB** worth of data **on disk**.

In [ ]:
%%time
it = 400
ca = bcolz.carray(a, rootdir='ca',
                 cparams=bcolz.cparams(clevel=9))
for i in range(it):
    ca.append(a)

The **compression ratio** in this case is again **130+**.

In [ ]:
ca

**Simple numerical operations** are easy to implement.

In [ ]:
%time np.sum(ca[:1000] + ca[4000:5000])

Let us try the previous, **mathematically more demanding operation** &ndash; again with a sub-set of the data.

In [ ]:
x = ca[:10000]  # 10,000 rows as sub-set

First, with an **in-memory ``carray`` results object**.

In [ ]:
%time bcolz.eval('x ** 2 + sqrt(x)', cparams=bcolz.cparams(9))

Second, with an **on-disk results object**. The time difference is not that huge.

In [ ]:
%time bcolz.eval('x ** 2 + sqrt(x)', cparams=bcolz.cparams(9), rootdir='out')

Finally, we verify **system disk usage**.

In [ ]:
!du -hs ca
# system disk usage

In [ ]:
!du -hs out

In [ ]:
!rm -r ca
!rm -r out

## blaze &ndash; Data Blending and Analysis

``blaze`` allows Python users a familiar interface to **query data living in diverse data storage systems**.
Cf. <a href="http://blaze.pydata.org/" target="_blank">http://blaze.pydata.org/</a>.

In [ ]:
import blaze as bz

### Simple Example 

The first example constructs a ``blaze.Data`` object from native Python objects.

In [ ]:
t = bz.Data([('Henry', 'boy', 8),
              ('Lilli', 'girl', 14)],
            fields=['name', 'gender', 'age'])

In [ ]:
t

In [ ]:
t[t.age > 10]

### Data from NumPy Array

Let us read data from an **in-memory NumPy ``ndarray`` object**.

In [ ]:
import numpy as np

In [ ]:
a = np.random.standard_normal((100000, 5))
  # 100,000 data rows, 5 columns

In [ ]:
df = bz.DataFrame(a, columns=['f0', 'f1', 'f2', 'f3', 'f4'])
  # blaze DataFrame constructor

A look at the **data structure**.

In [ ]:
df.head()

Data itself is stored as **NumPy ``ndarray`` object**.

In [ ]:
df.values

### Data from CSV File

We generate first a **CSV file** using the random data from before.

In [ ]:
%time df.to_csv('data.csv', index=False)

Let us **read the data** with ``blaze``. Actually, we **only generate a view**.

In [ ]:
%time csv = bz.CSV('data.csv')

In [ ]:
%time t1 = bz.Data(csv)

Now, we can **work** with the data. Note, however, that iterating, slicing, etc. are **not** (yet) implemented. 

In [ ]:
%time t1.count()

The **backend** is a **CSV object**. And a look at the **first 10 rows**.

In [ ]:
t1.data

In [ ]:
t1

### Data from SQL

We now generate a **SQLite3 table** with the dummy data from before.

In [ ]:
import sqlite3 as sq3

In [ ]:
con = sq3.connect('data.sql')
try:
    con.execute('DROP TABLE numbers')
    # delete in case it exists
except:
    pass

We **write the data** into an appropriate table.

In [ ]:
con.execute(
    'CREATE TABLE numbers (f0 real, f1 real, f2 real, f3 real, f4 real)'
    )

In [ ]:
%time con.executemany('INSERT INTO numbers VALUES (?, ?, ?, ?, ?)', a)

In [ ]:
con.commit()

In [ ]:
con.close()

Now **reading the data** with ``blaze`` (i.e. just generating a view).

In [ ]:
%time t2 = bz.Data('sqlite:///data.sql::numbers')

The **schema** and **first 10 rows** again.

In [ ]:
t2.schema

In [ ]:
t2

### Working with the blaze Objects

``blaze`` provides an **abstraction logic** for computations/queries.

In [ ]:
ts = bz.TableSymbol('ts',
        '{f0: float64, f1: float64, f2: float64, f3: float64, f4: float64}')
  # generic table description -- independent of the target data structure
expr = ts[ts['f0'] + ts['f3'] > 2.5]['f1']
  # generic expression -- independent of the target data structure

The ``blaze`` compiler **specializes the generic objects** to different data structures.

In [ ]:
%time np.array(bz.compute(expr, a))  # NumPy ndarray object

In [ ]:
%time np.array(bz.compute(expr, df))  # DataFrame object

In [ ]:
%time np.array(bz.compute(expr, csv))  # CSV file representation

In similar fashion, ``blaze`` allows **unified expression evaluations** for different backends (I).

In [ ]:
%time t1[t1['f0'] + t1['f3'] > 2.5]['f1'].head()
  # table representation 1
  # from CSV

In similar fashion, ``blaze`` allows **unified expression evaluations** for different backends (II).

In [ ]:
%time t2[t2['f0'] + t2['f3'] > 2.5]['f1'].head()
  # table representation 2
  # from SQL database

Typical **aggregational operations** work as well.

In [ ]:
%time t1.f0.sum()

In [ ]:
%time t2.f3.max()

### Transforming Data Formats

If you work intensively with data sets, it might be beneficial to transform them once into **highly performant binary data formats (eg ``bcolz, HDF5``)**.

#### Using bcolz as Data Store

In [ ]:
%time bz.into('data.bcolz', 'data.csv')
  # natively done by blaze
  # cparams=bcolz.cparams(9) could be added
  # no effect here due to random floats

We can now connect to the ``bcolz`` **disk-based ``ctable`` object**.

In [ ]:
import bcolz as bc

In [ ]:
b = bc.ctable(rootdir='data.bcolz') 

Now, the power of ``bcolz`` for **numerical computations** can be played out.

In [ ]:
%time nex = b.eval('sqrt(abs(f0)) + log(abs(f1))')

In [ ]:
nex

#### Using HDF5

Similarly, we can use **``PyTables`` and ``HDF5``** as an efficient binary store.

In [ ]:
import pandas as pd

In [ ]:
%%time
con = sq3.connect('data.sql')
pd.HDFStore('data.h5')['sql'] = pd.read_sql('SELECT * FROM numbers', con)
  # simultaneously reading whole SQL table and writing it to HDF5 store
con.close()

Now, data can be **efficiently** retrieved.

In [ ]:
%%time
%matplotlib inline
pd.HDFStore('data.h5')['sql'][::1000].cumsum().plot(figsize=(10, 5))
  # simultaneously reading data from HDF5 store and plotting it

### Cleaning Up

In [ ]:
!du -h dat*

In [ ]:
!ls -n dat*.*

In [ ]:
# cleaning up
!rm -r dat*.*

## Conclusion

**High performance (hardware-bound) I/O operations and highly efficient data blending and analytics are among Python's key strengths.**

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@pqp.io">team@tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**datapark.io** |
<a href="http://datapark.io">http://datapark.io</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>